In [ ]:
# 环境设置
import sys
import os
from pathlib import Path

# 添加项目路径
project_root = Path.cwd()  # 假设当前在项目根目录
if 'app' not in sys.path:
    sys.path.insert(0, str(project_root))

# 导入必要的库
import numpy as np
import json
from datetime import datetime
from typing import List, Dict, Optional
import traceback
import logging

# 导入工作流相关模块
from app.core.data.workflow_JINGFANG import DataQaWorkflow, WorkflowConfig
from app.models import (
    DataQACompletionRequest,
    DataQAChatCompletionResponse,
    ChatMessage as ModelChatMessage
)
from czce_ai.llm.chat import LLMChat as LLMModel
from czce_ai.llm.message import Message as ChatMessage
from czce_ai.utils.log import logger

# 设置日志级别
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

print("环境设置完成")
print(f"项目根目录: {project_root}")
print(f"Python路径: {sys.path[:3]}")  # 只显示前3个路径

In [ ]:
# 初始化LLM模型
from app.core.components import qwen3_llm, qwen3_thinking_llm

# 注意：如果遇到导入错误，可能需要直接创建LLM实例
try:
    ans_llm = qwen3_llm
    ans_thinking_llm = qwen3_thinking_llm
    query_llm = qwen3_llm
    print("使用预定义的LLM实例")
except:
    # 备用方案：直接创建LLM实例
    from app.config.config import settings
    ans_llm = LLMModel(model_name="qwen3")
    ans_thinking_llm = LLMModel(model_name="qwen3-thinking")
    query_llm = LLMModel(model_name="qwen3")
    print("创建新的LLM实例")

# 配置参数
workflow_config = WorkflowConfig(
    history_round=2,
    follow_up_round=1,
    reranking_threshold=0.2,
    collection="hybrid_sql",
    domain_collection="domain_kl",
    max_table_results=3,
    enable_entity_recognition=True,
    enable_reranker=True
)

# 初始化工作流 - 使用扩展参数
workflow = DataQaWorkflow(
    ans_llm=ans_llm,
    ans_thinking_llm=ans_thinking_llm,
    query_llm=query_llm,
    config=workflow_config,
    history_round=2,
    follow_up_round=1,
    reranking_threshold=0.2,
    knowledge_id="3cc33ed2-21fb-4452-9e10-528867bd5f99",
    bucket_name="czce-ai-dev",
    collection="hybrid_sql",
    use_cache=True
)

print("工作流初始化完成")
print(f"FAQ数据已加载: {len(workflow.faq_data)}条")
print(f"缓存文件位置: {workflow.cache_file}")

In [ ]:
# 基于您的FAQ知识库构造不同相似度的测试查询
# 注意：高相似度阈值是0.9，不是0.85

test_queries = {
    # 高相似度查询 (>=0.9) - 几乎完全匹配FAQ
    "high_similarity": [
        "查询郑商所当前主力合约的代码、结算价及成交量",  # 完全匹配
        "筛选当前主力合约结算价超过2000元/吨的品种",    # 完全匹配
        "棉花看涨期权主力合约持仓量占总持仓量的比例是多少", # 完全匹配
    ],
    
    # 中等相似度查询 (0.7-0.9) - 会作为参考
    "medium_similarity": [
        "查询郑商所主力合约的价格和成交情况",  # 相似但不完全相同
        "主力合约结算价超过2000的品种",       # 简化版本
        "棉花期权的持仓占比",                 # 更简短的表述
    ],
    
    # 低相似度查询 (<0.7) - 走完整流程
    "low_similarity": [
        "统计上个月的期货交易总额",
        "分析期货市场的风险指标",
        "计算所有交易所的平均手续费",
    ],
    
    # 需要追问的查询
    "follow_up_needed": [
        "查询成交量",      # 缺少品种和时间
        "统计持仓量",      # 缺少详细信息
        "最近的交易数据",  # 模糊查询
    ],
}

print("测试数据构造完成")
for category, queries in test_queries.items():
    print(f"{category}: {len(queries)}个查询")

In [ ]:
def test_entity_recognition():
    """测试增强版实体识别功能（包含合约代码标准化）"""
    print("=" * 50)
    print("🔍 测试 entity_recognition 函数")
    print("=" * 50)
    
    test_queries = [
        "查询FG2509的成交量",        # 标准合约代码
        "查询FG-2509的成交量",       # 带分隔符的合约代码
        "查询fg_2509的成交量",       # 小写+下划线
        "棉花CF2409合约的价格",
        "苹果期货AP2501的持仓量",
        "查询白糖SR的交易情况",
    ]
    
    for query in test_queries:
        try:
            enhanced = workflow.entity_recognition(query)
            print(f"原始: {query}")
            print(f"增强: {enhanced}")
            
            # 检查合约代码是否被正确标准化
            if "FG" in query.upper():
                assert "FG2509(合约)" in enhanced or "FG2509 (合约)" in enhanced, "合约代码未正确标准化"
            
            print("通过")
            print("-" * 30)
        except Exception as e:
            print(f"错误处理 '{query}': {e}")
    
    return True

# 执行测试
test_entity_recognition()

In [ ]:
def test_semantic_search_faq():
    """测试FAQ语义搜索功能"""
    print("=" * 50)
    print("🔎 测试 semantic_search_faq 函数")
    print("=" * 50)
    
    # 检查FAQ数据是否已加载
    if not workflow.faq_data:
        print("⚠️ FAQ数据未加载，尝试重新加载...")
        workflow._load_faqs()
    
    print(f"FAQ数据条数: {len(workflow.faq_data)}")
    
    # 测试不同相似度的查询
    test_cases = [
        ("查询郑商所当前主力合约的代码、结算价及成交量", "高相似度(>=0.9)"),
        ("查询主力合约的价格信息", "中等相似度(0.7-0.9)"),
        ("计算年度总收益", "低相似度(<0.7)"),
    ]
    
    for query, expected_similarity in test_cases:
        try:
            # 先进行实体识别
            enhanced_query = workflow.entity_recognition(query)
            
            # 执行FAQ搜索
            results = workflow.semantic_search_faq(enhanced_query, top_k=3)
            
            print(f"查询: {query}")
            print(f"增强: {enhanced_query}")
            print(f"期望: {expected_similarity}")
            
            if results:
                best_match = results[0]
                print(f"最佳匹配:")
                print(f"  问题: {best_match['question'][:50]}...")
                print(f"  相似度: {best_match['similarity']:.3f}")
                print(f"  表: {best_match['table']}")
                
                # 验证相似度范围
                sim = best_match['similarity']
                if sim >= 0.9:
                    actual = "高相似度(>=0.9)"
                elif sim >= 0.7:
                    actual = "中等相似度(0.7-0.9)"
                else:
                    actual = "低相似度(<0.7)"
                print(f"  实际: {actual}")
            else:
                print("  未找到相关FAQ")
            
            print("-" * 30)
            
        except Exception as e:
            print(f"❌ 错误: {e}")
            traceback.print_exc()
    
    return True

# 执行测试
test_semantic_search_faq()

In [ ]:
def test_e2e_high_similarity():
    """
    端到端测试：高相似度FAQ路径 (>=0.9)
    注意：阈值是0.9，不是0.85
    """
    print("=" * 80)
    print("🚀 端到端测试：高相似度FAQ路径 (>=0.9)")
    print("=" * 80)
    
    # 使用与FAQ完全匹配的查询
    user_query = "查询郑商所当前主力合约的代码、结算价及成交量"
    print(f"📝 用户查询: {user_query}")
    print("-" * 80)
    
    messages = [ChatMessage(role="user", content=user_query)]
    request = DataQACompletionRequest(
        messages=messages,
        model="test",
        created=int(datetime.now().timestamp()),
        follow_up_num=0,
        knowledge_base_ids=["3cc33ed2-21fb-4452-9e10-528867bd5f99"],
        use_reranker=True
    )
    
    try:
        print("⚙️ 执行do_generate函数...")
        response = workflow.do_generate(
            request=request,
            enable_follow_up=False,
            thinking=False
        )
        
        # 展示执行路径
        print("\n📊 执行路径:")
        for step in response.steps:
            print(f"  Step {step.number}: {step.name} [{step.key}]")
            if step.key == "semantic_search_faq":
                print(f"    ➜ {step.prompt}")
        
        # 检查是否走了FAQ快速路径
        print("\n💎 响应分析:")
        if response.model == "faq":
            print("  ✅ 模型类型: FAQ (快速返回)")
        else:
            print(f"  ⚠️ 模型类型: {response.model}")
        
        if len(response.steps) == 3:
            print("  ✅ 步骤数: 3 (FAQ快速路径)")
        else:
            print(f"  ⚠️ 步骤数: {len(response.steps)}")
        
        # 展示SQL输出
        if response.choices:
            content = response.choices[0].message.content
            if "sql" in content.lower():
                print("\n📄 SQL输出:")
                print(content[:500])
        
        return response
        
    except Exception as e:
        print(f"❌ 错误: {e}")
        traceback.print_exc()
        return None

# 执行测试
high_sim_response = test_e2e_high_similarity()

In [ ]:
def test_e2e_medium_similarity():
    """
    端到端测试：中等相似度FAQ路径 (0.7-0.9)
    FAQ作为参考，走完整6步流程
    """
    print("=" * 80)
    print("端到端测试：中等相似度FAQ路径 (0.7-0.9)")
    print("=" * 80)
    
    user_query = "查询郑商所主力合约的价格信息"
    print(f"用户查询: {user_query}")
    print("-" * 80)
    
    messages = [ChatMessage(role="user", content=user_query)]
    request = DataQACompletionRequest(
        messages=messages,
        model="test",
        created=int(datetime.now().timestamp()),
        follow_up_num=0,
        knowledge_base_ids=["3cc33ed2-21fb-4452-9e10-528867bd5f99"],
        use_reranker=True
    )
    
    try:
        print("执行do_generate函数...")
        response = workflow.do_generate(
            request=request,
            enable_follow_up=False,
            thinking=False
        )
        
        # 展示完整执行路径
        print("\n执行路径:")
        expected_steps = ["modify_query", "entity_recognition", "semantic_search_faq", 
                         "locate_table", "generate_sql"]
        
        for step in response.steps:
            print(f"  Step {step.number}: {step.name} [{step.key}]")
            if step.key in expected_steps:
                expected_steps.remove(step.key)
        
        # 验证是否走了完整流程
        print("\n路径验证:")
        if len(response.steps) == 5:
            print(" 步骤数: 5 (完整流程)")
            print(" FAQ作为参考添加到prompt")
        else:
            print(f" 步骤数: {len(response.steps)}")
        
        if not expected_steps:
            print("  所有预期步骤都已执行")
        else:
            print(f" 缺少步骤: {expected_steps}")
        
        return response
        
    except Exception as e:
        print(f"错误: {e}")
        traceback.print_exc()
        return None

# 执行测试
medium_sim_response = test_e2e_medium_similarity()

In [ ]:
def test_e2e_low_similarity():
    """
    端到端测试：低相似度路径 (<0.7)
    无FAQ参考，走完整6步流程
    """
    print("=" * 80)
    print("🔧 端到端测试：低相似度路径 (<0.7)")
    print("=" * 80)
    
    user_query = "统计上个月所有交易所的期货总交易额"
    print(f"📝 用户查询: {user_query}")
    print("-" * 80)
    
    messages = [ChatMessage(role="user", content=user_query)]
    request = DataQACompletionRequest(
        messages=messages,
        model="test",
        created=int(datetime.now().timestamp()),
        follow_up_num=0,
        knowledge_base_ids=["3cc33ed2-21fb-4452-9e10-528867bd5f99"],
        use_reranker=True
    )
    
    try:
        print("⚙️ 执行do_generate函数...")
        response = workflow.do_generate(
            request=request,
            enable_follow_up=False,
            thinking=False
        )
        
        # 展示执行路径
        print("\n📊 执行路径:")
        has_faq_reference = False
        
        for step in response.steps:
            print(f"  Step {step.number}: {step.name} [{step.key}]")
            if step.key == "semantic_search_faq":
                if "没有找到" in str(step.prompt) or "不满足" in str(step.prompt):
                    print(f"    ➜ FAQ相似度过低，不作为参考")
            elif step.key == "generate_single_table_prompt":
                # 检查prompt中是否有FAQ参考
                if "参考示例" in str(step.prompt):
                    has_faq_reference = True
        
        # 验证路径
        print("\n💎 路径验证:")
        if len(response.steps) == 6 and not has_faq_reference:
            print("  ✅ 步骤数: 6 (完整流程)")
            print("  ✅ 无FAQ参考（相似度<0.7）")
            print("  ✅ 纯LLM生成SQL")
        else:
            print(f"  ⚠️ 步骤数: {len(response.steps)}")
            if has_faq_reference:
                print("  ⚠️ 意外包含FAQ参考")
        
        return response
        
    except Exception as e:
        print(f"❌ 错误: {e}")
        traceback.print_exc()
        return None

# 执行测试
low_sim_response = test_e2e_low_similarity()